In [ ]:
import redcap
import pandas as pd
from temba_client.v2 import TembaClient
from temba_client.exceptions import TembaRateExceededError, TembaTokenError, TembaHttpError, TembaNoSuchObjectError
import math
import numpy as np


Define classes to contain the different methods that will be used

In [ ]:
class RedCap:
    def read_redcap_credentials_file(self,credential_file,form):
        with open(credential_file, encoding='utf-8') as data_file:
            data = json.loads(data_file.read())
        self.redcap_token = data['redcap_token']
        self.redcap_url = data['redcap_url']
        self.form = form
        
    
    def get_metadata(self):
        payload_metadata = {
            'token': self.redcap_token,
            'content': 'metadata',
            'format': 'json',
            'type': 'eav',
            'rawOrLabel': 'raw',
            'rawOrLabelHeaders': 'raw',
            'exportCheckboxLabel': 'false',
            'exportSurveyFields': 'false',
            'exportDataAccessGroups': 'false',
            'returnFormat': 'json',
            'forms': self.form
        }

        data_metadata = post(self.redcap_url, data=payload_metadata)
        metadata_df = pd.DataFrame.from_dict(data_metadata.json())
        metadata_df['branch_variable'] = metadata_df['branching_logic'].str.extract('((?<=\[).+?(?=\]))', expand=True)
        metadata_df['branch_variable'] =metadata_df['branch_variable'].str.replace('(','___').str.replace(')','')
        metadata_df['branch_value'] = metadata_df['branching_logic'].str.extract('((?<=\').*?(?=\'))', expand=True)
        metadata_df = metadata_df[metadata_df['form_name']==self.form]
        self.metadata_df = metadata_df
        return(metadata_df)
    
    def get_records(self):
        payload_records = {
            'token': self.redcap_token,
            'content': 'record',
            'format': 'json',
            'type': 'eav',
            'rawOrLabel': 'Label',
            'rawOrLabelHeaders': 'Label',
            'exportCheckboxLabel': 'false',
            'exportSurveyFields': 'true',
            'exportDataAccessGroups': 'false',
            'returnFormat': 'json',
            'forms': self.form
        }

        data_records = post(self.redcap_url, data=payload_records)
        records_df = pd.DataFrame.from_dict(data_records.json())
        n_records = np.int(np.max(records_df['record'])) + 1
        list_records = []
        for i in range(1,n_records):
            rr = records_df[records_df['record']==str(i)].T
            rr = rr.rename(columns = rr.iloc[0]).drop(['record','field_name'])
            r_dict = rr.to_dict('records')[0]
            list_records.append(r_dict)
        self.report_df = pd.DataFrame.from_dict(list_records)
        return(self.report_df)
    
    def get_expected_fields(self):
        branch_vals = self.metadata_df[['branch_variable','branch_value','field_name']]
        report_df = self.report_df
        expected_fields = []
        for indext,rowt in data_records.iterrows():
            expected_fields_temp = []
            report_df = rowt   
            for index,row in branch_vals.iterrows():
                if str(row['branch_variable']) =='nan': #if the branch variable is nan then we append this field
                    expected_fields_temp.append(row['field_name']) 
                if str(row['branch_variable']) !='nan': #if the branch variable is not nan then we need to check if the branch variable matached
                    if str(row['branch_variable']) in report_df:
                        if str(report_df[str(row['branch_variable'])][0]) == str(row['branch_value']) :
                            expected_fields_temp.append(row['field_name'])
            expected_fields.append(expected_fields_temp)
        self.expected_fields = expected_fields
        return(expected_fields)
    
    def get_completed_fields(self):
        data_records = self.report_df
        completed_fields_list = []
        for index,row in data_records.iterrows():
            completed = data_records.loc[index][~data_records.loc[index].isna()].index.tolist()
            completed_fields_list.append(completed)
        self.completed_fields_list = completed_fields_list
        return(completed_fields_list)
    
    def get_difference_fields(self):
        expected = self.get_expected_fields()
        completed = self.get_completed_fields()
        difference = []
        for i in range(0,2):
            difference.append(list(set(expected[i]).difference(completed[i])))
        self.difference = difference
        return(difference)
            


class rapidPro:
    def read_rapidpro_credentials_file(self,credential_file,rapidpro_flow_id):
        with open(credential_file, encoding='utf-8') as data_file:
            data = json.loads(data_file.read())
        self.rapidpro_apikey = data['rapidpro_apikey']
        self.rapidpro_url = data['rapidpro_url']
        self.client = TembaClient(self.rapidpro_url, self.rapidpro_apikey)
        self.rapidpro_message_sending_flow = rapidpro_flow_id
    
    def add_contact_to_groups(self,contact_urn, group_names):
        
        #get the groups we need to add them too
        additional_groups = self.client.get_groups(name = group_names)
        #check if contact exists first
        contacts = self.client.get_contacts(urn = contact_urn)
        for contact in contacts.all():
            current_groups_dynamic = contact.groups
            current_groups = self.client.get_groups(contact.groups)
            #this gets a little weird before it gets better
            gg = [] #first establish a blank list to store the uuids of the groups 
            if len(contact.groups) > 0 : #only require this if there were groups associated with the contact before
                for g in current_groups.all(): #here we append all the existing uuids of groups that the contact is associated with
                    gg.append(g.uuid)
            for g in additional_groups.all(): #here we add the uuids of the additional groups that are needed
                gg.append(g.uuid)
            contact = self.client.update_contact(contact_urn,groups = gg)
    
    def add_fields_to_contact(self,contact_urn, field_dict):
        client = TembaClient(self.rapidpro_url, self.rapidpro_apikey)
        #get the groups we need to add them too
        add_fields = client.update_contact(contact_urn, fields = field_dict)
    
    def start_flow(self,contact_urn):
        client = TembaClient(self.rapidpro_url, self.rapidpro_apikey)
        #get the groups we need to add them too
        client.create_flow_start(self.rapidpro_message_sending_flow, urns=[contact_urn])


        
        

In [ ]:
from requests import post
redcap = RedCap()
rapidpro = rapidPro()

In [ ]:
redcap.read_redcap_credentials_file(credential_file = 'redcapcredentials.json',form = 'asos2_investigator_phase1')
rapidpro.read_rapidpro_credentials_file(credential_file = 'rapidprocredentials.json',rapidpro_flow_id = 'bcaf334d-0fe0-4b45-8adb-1647aa3318c5')

Get the record data and metadata

In [ ]:
data_records = redcap.get_records()
data_metadata = redcap.get_metadata()
difference = redcap.get_difference_fields()


In [ ]:
redcap.form

Next we need to parse the phone number from the Redcap database and check it works -- it needs to be converted into the correct format for the message sender

In [ ]:
import phonenumbers
import types

for index,row in data_records.iterrows():
    wa_number = row['whatsapp_number']
    f_wa_number = phonenumbers.parse(wa_number, None)
    phone_number = ('tel:'+phonenumbers.format_number(f_wa_number,
                                                      phonenumbers.PhoneNumberFormat.E164))
    print(phone_number)
    contacts = rapidpro.client.get_contacts(urn = phone_number)
    #if the contact exists then make sure it is in the correct group
    if contacts.first() is None:
        print('contact doesnt exist- creating')
        rapidpro.client.create_contact(urns = [phone_number])
        rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
    else:
        print('contact already exists- just updating')
        rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
    number_missing_fields = len(difference[index])
    print(number_missing_fields)
    name = row['name']
    if(number_missing_fields > 0):
        string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. We have noticed that there are {missing_vals} missing fields in your latest survey. Would you mind completing the registration when you are able? Thanks!'.format(missing_vals=number_missing_fields,name=name)
    else:
        string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. You have completed the latest survey. Thanks!'.format(form_name = redcap.form,name=name)

    print(string)
    message_dict = {'wa_message' : string }
    rapidpro.add_fields_to_contact(phone_number,message_dict)
    rapidpro.start_flow(phone_number)

Here we update all the contacts as necessary